## Notebook in Snowpark Container Services 

### Import Libraries

In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Enable INFO log level
# import logging
# logging.basicConfig()
# logging.getLogger().setLevel(logging.INFO)

### Set Variables

In [2]:
SNOWFLAKE_DATABASE  = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA")
SNOWFLAKE_WAREHOUSE = 'DASH_L'

### Establish Secure Connection

In [3]:
# Read the login token supplied automatically by Snowflake. These tokens are short lived and should always be read right before creating any new connection.
def get_login_token():
  with open("/snowflake/session/token", "r") as f:
    return f.read()

# Construct Snowflake connection params from environment variables.
def get_connection_params():
  return {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "host": os.getenv("SNOWFLAKE_HOST"),
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": SNOWFLAKE_DATABASE,
    "schema": SNOWFLAKE_SCHEMA,
    "authenticator": "oauth",
    "token": get_login_token()
  }

# Create Snowflake Session object
session = Session.builder.configs(get_connection_params()).create()
session.sql_simplifier_enabled = True
snowpark_version = VERSION

# Current Environment Details
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowpark for Python version : 1.14.0


### Load Data

In [4]:
df = session.table('cell_towers_ca').order_by(F.col('TOWER_ID'))
df.show()

-----------------------------------------------------------------------------------------------------------------------------------
|"TOWER_ID"  |"TOWER_NAME"  |"LAT"    |"LON"      |"STATUS"  |"STATUS_MESSAGE"        |"LAST_COMM"          |"MAINTENANCE_DUE"    |
-----------------------------------------------------------------------------------------------------------------------------------
|1           |Tower 1       |34.1755  |-118.3372  |Online    |Online                  |2024-04-07 00:00:00  |2024-04-07 00:00:00  |
|2           |Tower 2       |34.0776  |-118.2376  |Online    |All systems normal      |2024-04-07 01:00:00  |2024-04-07 01:00:00  |
|3           |Tower 3       |37.7744  |-122.4128  |Offline   |Power failure detected  |2024-04-07 02:00:00  |2024-04-07 02:00:00  |
|4           |Tower 4       |37.7744  |-122.0724  |Online    |Online                  |2024-04-07 03:00:00  |2024-04-07 03:00:00  |
|5           |Tower 5       |37.603   |-122.0771  |Online    |All systems no